# 모듈 임포트

In [96]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import requests
from PIL import Image
import io
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# chromed driver 불러오기

In [97]:
os.chmod('/Users/jeongdaegyun/Desktop/Musinsa/selenium_crawling/chromedriver_mac_arm64/chromedriver', 0o755)

driver = webdriver.Chrome(executable_path="./chromedriver_mac_arm64/chromedriver")

# 무신사 코디맵으로 이동

In [98]:
driver.get("https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1")
driver.implicitly_wait(3)

# 데이터프레임의 한 행에 해당하는 샘플을 추출하는 함수

In [99]:
# num: 한 페이지의 코디는 1~60까지 인덱싱되어있음 
# img_name: 저장할 이미지의 이름
# path: 저장할 경로 (기본은 같은 경로의 image라는 폴더)

def get_sample(num, img_name = None, path = './image'):
    
    # sample이 담길 list
    data = []
    
    # 페이지 이동
    element = driver.find_element(By.XPATH, f'/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{num}]/div[2]/a/strong')
    driver.execute_script("arguments[0].click();", element)
    
    # 제목 추출
    title = driver.find_element(By.XPATH, '//*[@id="style_info"]/h2').text
    
    # 코디에 대한 설명 추출
    description = driver.find_element(By.XPATH, '//*[@id="style_info"]/div[1]/p').text
    
    
    if img_name == None:
        raise Exception('img_name 인자가 없습니다!')
    else:    
        #이미지 url 추출
        src = driver.find_element(By.CLASS_NAME, 'photo').get_attribute('src')

        # GET 요청으로 이미지 데이터 가져오기
        response = requests.get(src)

        # 가져온 데이터를 이미지 파일로 변환하기
        image = Image.open(io.BytesIO(response.content))

        # 이미지 파일로 저장하기
        image.save(path+'/'+img_name+'.jpg')
        
        driver.back()
        driver.implicitly_wait(3)
        data.append(title)
        data.append(path+'/'+img_name+'.jpg')
        data.append(description)
        
        return data

# 데이터 추출

In [100]:
page_num = 20 # 추출할 페이지 분량 (페이지 당 60개)

In [101]:
samples = []
for j in range(1, page_num+1):
    driver.get(f"https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page={j}")
    driver.implicitly_wait(3)
    for i in range(1, 61):    
        samples.append(get_sample(i, img_name = f'image_{(j-1)*60+i}'))
        time.sleep(1)

# ndarray로 바꾸어 형태 확인

In [102]:
samples = np.array(samples)

In [103]:
samples.shape

(1200, 3)

# dataframe에 넣기

In [104]:
df = pd.DataFrame(samples, columns = ['title', 'img_path', 'description'])

In [105]:
df.head(6)

,title,img_path,description
0,언제나 좋아,./image/image_1.jpg,그래픽 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 에코 백으로 마무리한 캐...
1,데일리로 즐겨,./image/image_2.jpg,컬러 배색이 돋보이는 집업과 데님 팬츠를 매치하고 크로스 백으로 완성한 캐주얼 룩
2,꿀 조합,./image/image_3.jpg,로고 포인트의 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 마무리한 캐주얼 룩
3,편안한 꾸안꾸,./image/image_4.jpg,로고 디테일이 돋보이는 후드 티셔츠와 스웨트 팬츠를 코디하고 볼캡으로 마무리한 캐주얼 룩
4,단정한 매력,./image/image_5.jpg,단정한 무드의 블레이저 재킷과 슬랙스를 매치하고 토트백을 더해 연출한 포멀 룩
5,디테일에 집중!,./image/image_6.jpg,컬러 배색이 돋보이는 레더 재킷과 부츠 컷 팬츠를 코디하고 비니로 멋을 더한 스트릿 룩


# 이미지 이름과 인덱스 일치시키기

In [106]:
df.index = [i+1 for i in df.index]

In [107]:
df.head()

,title,img_path,description
1,언제나 좋아,./image/image_1.jpg,그래픽 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 에코 백으로 마무리한 캐...
2,데일리로 즐겨,./image/image_2.jpg,컬러 배색이 돋보이는 집업과 데님 팬츠를 매치하고 크로스 백으로 완성한 캐주얼 룩
3,꿀 조합,./image/image_3.jpg,로고 포인트의 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 마무리한 캐주얼 룩
4,편안한 꾸안꾸,./image/image_4.jpg,로고 디테일이 돋보이는 후드 티셔츠와 스웨트 팬츠를 코디하고 볼캡으로 마무리한 캐주얼 룩
5,단정한 매력,./image/image_5.jpg,단정한 무드의 블레이저 재킷과 슬랙스를 매치하고 토트백을 더해 연출한 포멀 룩


# 결측치, 형태 확인

In [108]:
df.isnull().sum()

title          0
img_path       0
description    0
dtype: int64

In [109]:
df.shape

(1200, 3)

In [110]:
df.to_csv('data.csv', sep=',', na_rep='NaN')